# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd 

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
collections = db.list_collection_names()
pprint(collections)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"hygiene_score": 20}
results = db.establishments.find(query)

# Use count_documents to display the number of documents in the result
num_results = db.establishments.count_documents(query)
print("Number of documents with hygiene score of 20:", num_results)

# Display the first document in the results using pprint
result = db.find_one(query)
pprint(result)


In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthority": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
num_documents = db.count_documents(query)
print("Number of documents with London as Local Authority and RatingValue greather than or equal to 4:", num_documents)

# Display the first document in the results using pprint
result = db.find_one(query)
pprint(result)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.5074
longitude = 0.1278

query = {"latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
"longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
"RatingValue": 5}

sort = [("hygiene_score", 1)]

# Print the results
results = db.find(query).sort(sort)
for result in results:
    pprint(result)

In [ ]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(results))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [{"$match": {"hygiene_score": 0}},
{"$group": {"_id": "$LocalAuthority", "count": {"$sum": 1}}},
{"$sort": {"count": -1}}]

# Print the number of documents in the result
result = list(db.aggregate(pipeline))
num_documents = len(result)
print("Number of documents with hygiene score of 0", num_documents)

# Print the first 10 results
print("First 10 results:")
for i in range(min(10, num_documents)):
    pprint(result[i])

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(db.aggregate(pipeline)))

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))
